<a href="https://colab.research.google.com/github/Shreyojit/AI-Finance-Agent-/blob/main/Finance_Agent_using_LangChain_(LlaMa_3_%7C_Groq_API).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import yfinance as yf

yf.__version__

'0.2.50'

In [3]:

groq_api_key='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'

In [2]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25


In [10]:
from langchain_groq import ChatGroq

llama3 = ChatGroq(api_key=groq_api_key, model="llama3-70b-8192")

llama3

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fbfe6ef3ca0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fbfe6ef2410>, model_name='llama3-70b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

#Define Tools

In [8]:
from langchain_core.tools import tool, StructuredTool
from datetime import date

@tool
def company_information(ticker: str) -> dict:
    """Use this tool to retrieve company information like address, industry, sector, company officers, business summary, website,
       marketCap, current price, ebitda, total debt, total revenue, debt-to-equity, etc."""

    ticker_obj = yf.Ticker(ticker)
    ticker_info = ticker_obj.get_info()

    return ticker_info

@tool
def last_dividend_and_earnings_date(ticker: str) -> dict:
    """
    Use this tool to retrieve company's last dividend date and earnings release dates.
    It does not provide information about historical dividend yields.
    """
    ticker_obj = yf.Ticker(ticker)

    return ticker_obj.get_calendar()

@tool
def summary_of_mutual_fund_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top mutual fund holders.
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)
    mf_holders = ticker_obj.get_mutualfund_holders()

    return mf_holders.to_dict(orient="records")

@tool
def summary_of_institutional_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top institutional holders.
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)
    inst_holders = ticker_obj.get_institutional_holders()

    return inst_holders.to_dict(orient="records")

@tool
def stock_grade_updrages_downgrades(ticker: str) -> dict:
    """
    Use this to retrieve grade ratings upgrades and downgrades details of particular stock.
    It'll provide name of firms along with 'To Grade' and 'From Grade' details. Grade date is also provided.
    """
    ticker_obj = yf.Ticker(ticker)

    curr_year = date.today().year

    upgrades_downgrades = ticker_obj.get_upgrades_downgrades()
    upgrades_downgrades = upgrades_downgrades.loc[upgrades_downgrades.index > f"{curr_year}-01-01"]
    upgrades_downgrades = upgrades_downgrades[upgrades_downgrades["Action"].isin(["up", "down"])]

    return upgrades_downgrades.to_dict(orient="records")

@tool
def stock_splits_history(ticker: str) -> dict:
    """
    Use this tool to retrieve company's historical stock splits data.
    """
    ticker_obj = yf.Ticker(ticker)
    hist_splits = ticker_obj.get_splits()

    return hist_splits.to_dict()

@tool
def stock_news(ticker: str) -> dict:
    """
    Use this to retrieve latest news articles discussing particular stock ticker.
    """
    ticker_obj = yf.Ticker(ticker)

    return ticker_obj.get_news()


#stock_news_tool = StructuredTool.from_function(stock_news)

3.Define Agents(LLM+Tools)

In [23]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

tools = [
         company_information,
         last_dividend_and_earnings_date,
         stock_splits_history,
         summary_of_mutual_fund_holders,
         summary_of_institutional_holders,
         stock_grade_updrages_downgrades,
         stock_news
]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Try to answer user query using available tools.",
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llama3 = ChatGroq(api_key=groq_api_key, model="llama3-70b-8192", temperature=0)

finance_agent = create_tool_calling_agent(llama3, tools, prompt)

finance_agent_executor = AgentExecutor(agent=finance_agent, tools=tools, verbose=True)

In [11]:
from langchain_core.messages import HumanMessage

resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is address of microsoft?")]})



> Entering new AgentExecutor chain...

Invoking: `company_information` with `{'ticker': 'MSFT'}`


{'address1': 'One Microsoft Way', 'city': 'Redmond', 'state': 'WA', 'zip': '98052-6399', 'country': 'United States', 'phone': '425 882 8080', 'website': 'https://www.microsoft.com', 'industry': 'Software - Infrastructure', 'industryKey': 'software-infrastructure', 'industryDisp': 'Software - Infrastructure', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, includ

In [14]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is dividend yield for microsoft stock?")]})

In [16]:
print(resp["output"])

The dividend yield for Microsoft stock is 0.77%.


In [19]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="Summarize Key people at apple")]})

In [20]:

print(resp["output"])

Here is a summary of the key people at Apple:

1. **Timothy D. Cook** - CEO & Director, age 62
2. **Luca Maestri** - CFO & Senior VP, age 60
3. **Jeffrey E. Williams** - Chief Operating Officer, age 59
4. **Katherine L. Adams** - Senior VP, General Counsel & Secretary, age 59
5. **Deirdre O'Brien** - Chief People Officer & Senior VP of Retail, age 56
6. **Chris Kondo** - Senior Director of Corporate Accounting
7. **Suhasini Chandramouli** - Director of Investor Relations
8. **Greg Joswiak** - Senior Vice President of Worldwide Marketing
9. **Adrian Perica** - Head of Corporate Development, age 49
10. **Michael Fenger** - VP of Worldwide Sales


In [21]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="What was last dividend date for apple stock?")]})

In [22]:
print(resp["output"])

The last dividend date for Apple stock is November 14, 2024.


In [25]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="Top mutual fund holders of microsoft stock.")]})



> Entering new AgentExecutor chain...

Invoking: `summary_of_mutual_fund_holders` with `{'ticker': 'MSFT'}`


[{'Date Reported': Timestamp('2023-06-30 00:00:00'), 'Holder': 'Vanguard Total Stock Market Index Fund', 'pctHeld': 0.0312, 'Shares': 231883087, 'Value': 78965466446}, {'Date Reported': Timestamp('2023-06-30 00:00:00'), 'Holder': 'Vanguard 500 Index Fund', 'pctHeld': 0.0238, 'Shares': 177034936, 'Value': 60287477105}, {'Date Reported': Timestamp('2023-08-31 00:00:00'), 'Holder': 'Fidelity 500 Index Fund', 'pctHeld': 0.0115, 'Shares': 85181943, 'Value': 27919233637}, {'Date Reported': Timestamp('2023-09-30 00:00:00'), 'Holder': 'SPDR S&P 500 ETF Trust', 'pctHeld': 0.0112, 'Shares': 83514381, 'Value': 26369665800}, {'Date Reported': Timestamp('2023-09-30 00:00:00'), 'Holder': 'iShares Core S&P 500 ETF', 'pctHeld': 0.0095, 'Shares': 70264910, 'Value': 22186145332}, {'Date Reported': Timestamp('2023-06-30 00:00:00'), 'Holder': 'Vanguard Growth Index Fund', 'pctHeld': 0.0085, 'Sha

In [26]:
print(resp["output"])

Here are the top mutual fund holders of Microsoft stock:

1. Vanguard Total Stock Market Index Fund - 3.12% of shares held
2. Vanguard 500 Index Fund - 2.38% of shares held
3. Fidelity 500 Index Fund - 1.15% of shares held
4. SPDR S&P 500 ETF Trust - 1.12% of shares held
5. iShares Core S&P 500 ETF - 0.95% of shares held
6. Vanguard Growth Index Fund - 0.85% of shares held
7. Invesco ETF Tr-Invesco QQQ Tr, Series 1 ETF - 0.79% of shares held
8. Vanguard Institutional Index Fund-Institutional Index Fund - 0.68% of shares held
9. Vanguard Information Technology Index Fund - 0.53% of shares held
10. Growth Fund Of America Inc - 0.53% of shares held


In [27]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="Top institutional holders of microsoft stock.")]})



> Entering new AgentExecutor chain...

Invoking: `summary_of_institutional_holders` with `{'ticker': 'MSFT'}`


[{'Date Reported': Timestamp('2023-06-30 00:00:00'), 'Holder': 'Vanguard Group Inc', 'pctHeld': 0.0879, 'Shares': 653247546, 'Value': 222456919314}, {'Date Reported': Timestamp('2023-06-30 00:00:00'), 'Holder': 'Blackrock Inc.', 'pctHeld': 0.0722, 'Shares': 536245027, 'Value': 182612881494}, {'Date Reported': Timestamp('2023-06-30 00:00:00'), 'Holder': 'State Street Corporation', 'pctHeld': 0.0395, 'Shares': 293614078, 'Value': 99987338122}, {'Date Reported': Timestamp('2023-06-30 00:00:00'), 'Holder': 'FMR, LLC', 'pctHeld': 0.028299998, 'Shares': 210111250, 'Value': 71551285075}, {'Date Reported': Timestamp('2023-06-30 00:00:00'), 'Holder': 'Price (T.Rowe) Associates Inc', 'pctHeld': 0.0215, 'Shares': 159904219, 'Value': 54453782738}, {'Date Reported': Timestamp('2023-06-30 00:00:00'), 'Holder': 'Geode Capital Management, LLC', 'pctHeld': 0.0199, 'Shares': 148129470, 'Valu

In [28]:
print(resp["output"])

Here are the top institutional holders of Microsoft stock:

1. Vanguard Group Inc - 8.79% of shares held
2. Blackrock Inc. - 7.22% of shares held
3. State Street Corporation - 3.95% of shares held
4. FMR, LLC - 2.83% of shares held
5. Price (T.Rowe) Associates Inc - 2.15% of shares held
6. Geode Capital Management, LLC - 1.99% of shares held
7. Morgan Stanley - 1.66% of shares held
8. JP Morgan Chase & Company - 1.52% of shares held
9. Norges Bank Investment Management - 1.16% of shares held
10. Capital World Investors - 1.15% of shares held


In [29]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="Which firms recently changed grade of apple stock? Also, return rating changes.")]})



> Entering new AgentExecutor chain...

Invoking: `stock_grade_updrages_downgrades` with `{'ticker': 'AAPL'}`


[{'Firm': 'Keybanc', 'ToGrade': 'Underweight', 'FromGrade': 'Sector Weight', 'Action': 'down'}, {'Firm': 'Loop Capital', 'ToGrade': 'Buy', 'FromGrade': 'Hold', 'Action': 'up'}, {'Firm': 'Rosenblatt', 'ToGrade': 'Buy', 'FromGrade': 'Neutral', 'Action': 'up'}, {'Firm': 'Erste Group', 'ToGrade': 'Buy', 'FromGrade': 'Hold', 'Action': 'up'}, {'Firm': 'DA Davidson', 'ToGrade': 'Buy', 'FromGrade': 'Neutral', 'Action': 'up'}, {'Firm': 'Itau BBA', 'ToGrade': 'Market Perform', 'FromGrade': 'Underperform', 'Action': 'up'}, {'Firm': 'Bernstein', 'ToGrade': 'Outperform', 'FromGrade': 'Market Perform', 'Action': 'up'}, {'Firm': 'DZ Bank', 'ToGrade': 'Hold', 'FromGrade': 'Buy', 'Action': 'down'}, {'Firm': 'B of A Securities', 'ToGrade': 'Buy', 'FromGrade': 'Neutral', 'Action': 'up'}, {'Firm': 'Redburn Atlantic', 'ToGrade': 'Neutral', 'FromGrade': 'Buy', 'Action': 'down'}, {'Firm': 'Piper S

In [30]:
print(resp["output"])

Here is the list of firms that recently changed their grade of Apple stock, along with the rating changes:

* Keybanc: Downgraded from "Sector Weight" to "Underweight"
* Loop Capital: Upgraded from "Hold" to "Buy"
* Rosenblatt: Upgraded from "Neutral" to "Buy"
* Erste Group: Upgraded from "Hold" to "Buy"
* DA Davidson: Upgraded from "Neutral" to "Buy"
* Itau BBA: Upgraded from "Underperform" to "Market Perform"
* Bernstein: Upgraded from "Market Perform" to "Outperform"
* DZ Bank: Downgraded from "Buy" to "Hold"
* B of A Securities: Upgraded from "Neutral" to "Buy"
* Redburn Atlantic: Downgraded from "Buy" to "Neutral"
* Piper Sandler: Downgraded from "Overweight" to "Neutral"
* Barclays: Downgraded from "Equal-Weight" to "Underweight"
